Input Data here

In [ ]:
#Put in the real values for the concerned attribute wrt to batch size 
bs_1024 = 0
bs_256 = 0
bs_128 = 0
bs_32 = 0
bs_2 = 0
"""
Interpolation Method: 


Interpolation Methods List: 
1. linear 
2. barycentric 
3. krogh 
4. pchip 
5. cubicspline
6. spline, order = 2 - mention order in interpolate function e.g. d_interpolated=d_ground_truth.interpolate(method = interpolation_method, order = 2, limit_direction ='both') 
"""
interpolation_method = 'linear'

STEP 1: Creating the dataframe where we want to fill up missing values 

In [ ]:
import pandas as pd

d_ground_truth = pd.DataFrame({"Batch Size":[1024, 869, 716, 562, 409, 256, 230, 205, 180, 154, 128, 109, 90, 70, 51, 32, 26, 20, 14, 8, 2],
                   "Max. GPU Utilization percent":[bs_1024, None, None, None, None, bs_256, None, None, None, None, bs_128, None, None, None, None, bs_32, None, None, None, None, bs_2]})

# Print the dataframe
d_ground_truth

Batch Size  Max. GPU Utilization percent
0         1024                        40.870
1          869                           NaN
2          716                           NaN
3          562                           NaN
4          409                           NaN
5          256                        34.130
6          230                           NaN
7          205                           NaN
8          180                           NaN
9          154                           NaN
10         128                        29.530
11         109                           NaN
12          90                           NaN
13          70                           NaN
14          51                           NaN
15          32                        21.070
16          26                           NaN
17          20                           NaN
18          14                           NaN
19           8                           NaN
20           2                        18.633

STEP 2: to interpolate the missing values

In [ ]:
d_interpolated=d_ground_truth.interpolate(method = interpolation_method, limit_direction ='both') 

print(d_interpolated) #d_interpolated contains the synthetic and real values 


    Batch Size  Max. GPU Utilization percent
0         1024                       40.8700
1          869                       39.5220
2          716                       38.1740
3          562                       36.8260
4          409                       35.4780
5          256                       34.1300
6          230                       33.2100
7          205                       32.2900
8          180                       31.3700
9          154                       30.4500
10         128                       29.5300
11         109                       27.8380
12          90                       26.1460
13          70                       24.4540
14          51                       22.7620
15          32                       21.0700
16          26                       20.5826
17          20                       20.0952
18          14                       19.6078
19           8                       19.1204
20           2                       18.6330


STEP 3: extract synthetic values to a dataframe and keep real ones as 'None' 

In [ ]:
d_synthetic = d_interpolated
d_synthetic.loc[d_synthetic["Batch Size"] == 1024, "Max. GPU Utilization percent"] = float("nan")
d_synthetic.loc[d_synthetic["Batch Size"] == 256, "Max. GPU Utilization percent"] = float("nan")
d_synthetic.loc[d_synthetic["Batch Size"] == 128, "Max. GPU Utilization percent"] = float("nan")
d_synthetic.loc[d_synthetic["Batch Size"] == 32, "Max. GPU Utilization percent"] = float("nan")
d_synthetic.loc[d_synthetic["Batch Size"] == 2, "Max. GPU Utilization percent"] = float("nan")

#d_synthetic = pd.DataFrame({"Batch Size":[1024, 869, 716, 562, 409, 256, 230, 205, 180, 154, 128, 109, 90, 70, 51, 32, 26, 20, 14, 8, 2],
 #                  "Max. GPU Utilization percent":[None, 46.976, 47.882, 48.788, 49.694, None, 50.066, 49.532, 48.998, 48.464, None, 46.250, 44.570, 42.890, 41.210, None, 36.398, 33.266, 30.134, 27.002, None]})
d_synthetic

Batch Size  Max. GPU Utilization percent
0         1024                           NaN
1          869                       39.5220
2          716                       38.1740
3          562                       36.8260
4          409                       35.4780
5          256                           NaN
6          230                       33.2100
7          205                       32.2900
8          180                       31.3700
9          154                       30.4500
10         128                           NaN
11         109                       27.8380
12          90                       26.1460
13          70                       24.4540
14          51                       22.7620
15          32                           NaN
16          26                       20.5826
17          20                       20.0952
18          14                       19.6078
19           8                       19.1204
20           2                           NaN


STEP 4: use the synthetic values to re-generate real values 

In [ ]:
d_pseudo_real = d_synthetic.interpolate(method = interpolation_method, limit_direction ='both') 
d_pseudo_real

Batch Size  Max. GPU Utilization percent
0         1024                       39.5220
1          869                       39.5220
2          716                       38.1740
3          562                       36.8260
4          409                       35.4780
5          256                       34.3440
6          230                       33.2100
7          205                       32.2900
8          180                       31.3700
9          154                       30.4500
10         128                       29.1440
11         109                       27.8380
12          90                       26.1460
13          70                       24.4540
14          51                       22.7620
15          32                       21.6723
16          26                       20.5826
17          20                       20.0952
18          14                       19.6078
19           8                       19.1204
20           2                       19.1204

STEP 5: calculate the difference between ground truth and re-generated ground truth 

In [ ]:
y_true = [bs_1024, bs_128, bs_256, bs_32, bs_2]

bs_1024_generated = d_pseudo_real.loc[d_pseudo_real["Batch Size"] == 1024, "Max. GPU Utilization percent"] 
bs_256_generated = d_pseudo_real.loc[d_pseudo_real["Batch Size"] == 256, "Max. GPU Utilization percent"] 
bs_128_generated = d_pseudo_real.loc[d_pseudo_real["Batch Size"] == 128, "Max. GPU Utilization percent"] 
bs_32_generated = d_pseudo_real.loc[d_pseudo_real["Batch Size"] == 32, "Max. GPU Utilization percent"] 
bs_2_generated = d_pseudo_real.loc[d_pseudo_real["Batch Size"] == 2, "Max. GPU Utilization percent"] 

y_pred = [bs_1024_generated, bs_256_generated, bs_128_generated, bs_32_generated, bs_2_generated]



import sklearn
#Calculating Root Mean Square Error 
from sklearn.metrics import mean_squared_error
print("Root Mean Square Error (RMSE): ",mean_squared_error(y_true, y_pred, squared=False))


#Calculating Mean Square Error 
from sklearn.metrics import mean_squared_error
print("Mean Square Error (MSE): ", mean_squared_error(y_true, y_pred))


#Calculating Mean Absolute Error 
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error (MAE): ", mean_absolute_error(y_true, y_pred))

Root Mean Square Error (RMSE):  3.1765459244279786
Mean Square Error (MSE):  10.09044401
Mean Absolute Error (MAE):  2.44754
